## Задача - предсказать число книг на фотографии объявления
### Общее решение - будем использовать готовый детектор объектов с дополнительной фильтрацией, из которого будем считать число объектов с лейблом "book".
Почему не обучить свою модель? **Данные нельзя размечать**, поэтому для дообучения модели нужно сгенерировать хорошие синтетические данные. Я пробовал генерировать обложки книг и фонов с помощью Stable Diffusion 2, но результат очень слабо похож на реальные данные. К тому же, с учетом лимита по времени, намного проще и быстрее улучшить работу готового детектора, нежели дообучать модель.
### Общий пайплайн:
1. Изображение препроцессится - увеличивает контраст, убираем шум.
2. Изображение отправляется в детектор объектов D-Fine https://github.com/Peterande/D-FINE (ICLR Spotlight 2025 и совсем недавно добавили на HF). 
Выбор модели основан на ее быстрого инференса, а также хорошего качества предсказания и быстрой установки из HF. Модель предсказывает все объекты с порогом 0.
3. Изображение также отправляется в CLIP для дополнительной проверки наличия книг на фото. CLIP оценивает вероятность наличия книги на изображении.
4. Полученные предсказания от детектора и CLIP переходят в count_from_raw(), где происходит фильтрация по порогу. 
    a) Оставляем только объекты с лейблом книга.
    b) Если вероятность нахождения книг на картинке из CLIP меньше 0.05, то сразу возвращаем что число книг равно 0. Это делается как начальная фильтрация, в случаях когда детектор находит книги, где их нет.
    с) Фильтруем по верхнему порогу для детектора 0.6. Если нету минимум 3 объектов с таким скором, то понижаем его до нижней границы 0.4. Это делается специально для случаев, когда есть много книг, или целая стопка. В таком случае у детектора несколько ниже скоры для всех книг, и таким образом мы адаптивно понижаем порог. В случае с одной книгой, обычно есть только один большой скор, поэтому понижение порога ничего не меняет.  
    d) Если в результате фильтрации по результатам предсказаний детектора ничего нету, но CLIP говорит иначе, то как финальная проверка понижаем порог до 0.15 для детектора и пробуем найти книги.
    e) Наконец проводим NMS чтобы удалить избыточные пересекающиеся bboxы
    f) Возвращаем оставшиеся число объектов на картинке

### Что пробовал, но не сработало:
* Создание синтетического датасета с Stable Diffusion 2
* Детекция объектов по тайлам изображения
* Отдельная проверка на существование книги на каждом bboxe с помощью CLIP
* Ансамбль нескольких детекторов

### Результаты:
Модель предсказывает число книг с balanced accuracy ~50%. Метрика довольно жестокая, так как на паре примеров 20 книгами модель может ошибиться на 1-2 книги и все равно ответ будет считаться полностью неверным. На большинстве случаев (1 книга или 0 книг) модель показывает очень хорошие результаты. Тем не менее, результат можно улучшить, например, созданием нормального размеченного датасета и дообучения модели детектора на конкретных примерах изображений с книгами. Также можно использовать фичи детектора для обучения простого регрессора, нежели использовать число объектов напрямую. Можно улучшить фильтрацию изображений заменой CLIP на современные VLM модели, и возможно даже получать в качестве предсказаний конкретные числа книг, нежели их наличие/отсутствие. На этапе предсказания можно попробовать аугментации изображений во время инференса, так как некоторые изображения были изначально перевернуты, и поэтому могли быть не корректно анализированы.

In [ ]:
!pip install torch torchvision torchaudio
!pip install numpy pandas tqdm pillow scikit-learn matplotlib opencv-python
!pip install transformers

In [6]:
import os
import torch
import numpy as np
import pandas as pd
from glob import glob
from PIL import Image, ImageEnhance
from tqdm.auto import tqdm
from sklearn.metrics import balanced_accuracy_score
from torchvision.ops import nms
from transformers import AutoProcessor, DFineForObjectDetection, CLIPProcessor, CLIPModel
import matplotlib.pyplot as plt
import cv2


In [7]:
# variables
device = "cuda" if torch.cuda.is_available() else "cpu"
dataset_folder = "task_images"

# CLIP params
CLIP_MODEL_ID = "openai/clip-vit-base-patch32"
CLIP_TEXTS = ["a photo of a book", "no book"]
CLIP_PRESENCE_TH = 0.6

# detector params
detector_id = "ustc-community/dfine-xlarge-coco"
DETECTOR_THRESH = 0.6
DETECTOR_THRESH_LOW = 0.4

In [8]:
# detector wrapper
class DFineWrapper:
    def __init__(self, model_id):
        self.processor = AutoProcessor.from_pretrained(model_id)
        self.model = DFineForObjectDetection.from_pretrained(model_id).to(device)
        self.id2label = self.model.config.id2label
        inv = {v: k for k, v in self.id2label.items()}
        self.book_label_id = inv.get("book", None)

    # generate bboxes, scores and labels for given image with a given minimal score threshold 
    def predict_raw(self, pil_img, min_threshold=0.0):
        w, h = pil_img.size
        inputs = self.processor(images=pil_img, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = self.model(**inputs)
        target_sizes = [(h, w)]
        preds = self.processor.post_process_object_detection(
            outputs, target_sizes=target_sizes, threshold=min_threshold
        )
        if len(preds) == 0:
            return np.zeros((0, 4)), np.zeros(0), np.zeros(0, dtype=int)
        p = preds[0]
        boxes = p.get("boxes", torch.zeros((0, 4))).cpu().numpy()
        scores = p.get("scores", torch.zeros(0)).cpu().numpy()
        labels = p.get("labels", torch.zeros(0)).cpu().numpy().astype(int)
        return boxes, scores, labels

In [ ]:
# helper functions

# preprocess image before running models
def preprocess_image(image):
    np_img = np.array(image)

    # fast denoise
    np_img = cv2.fastNlMeansDenoisingColored(np_img, None, 10, 10, 7, 15)
    # image = ImageEnhance.Brightness(image).enhance(1.5)
    
    # increase contrast
    image = ImageEnhance.Contrast(image).enhance(1.2)
    return image

# perform nms to remove overlapping bboxes
def perform_nms_np(boxes, scores, iou_thresh):
    if boxes.shape[0] == 0:
        return np.array([], dtype=int)
    boxes_t = torch.from_numpy(boxes).float()
    scores_t = torch.from_numpy(scores).float()
    keep = nms(boxes_t, scores_t, iou_thresh).cpu().numpy()
    return keep

# function for running predictions on CLIP
def clip_image_book_prob(pil_img, clip_processor, clip_model):
    inputs = clip_processor(
        text=CLIP_TEXTS, images=pil_img, return_tensors="pt", padding=True
    ).to(device)
    with torch.no_grad():
        out = clip_model(**inputs)
    probs = torch.softmax(out.logits_per_image, dim=1).cpu().numpy()[0]
    # find logit index of a book being on a photo
    try:
        idx = CLIP_TEXTS.index("a photo of a book")
    except ValueError:
        raise RuntimeError(f"'book' not found in CLIP_TEXTS: {CLIP_TEXTS}")
    
    # return confidence of book being on a photo
    return float(probs[idx])

# main function for running detector and CLIP on every image
def build_predictions_dataset(wrapper, clip_processor, clip_model, folder, min_threshold=0.0, max_images=None, visualize_n=3):
    
    # get sorted paths of images 
    paths = sorted(
        glob(os.path.join(folder, "*.jpg")),
        key=lambda x: int(os.path.splitext(os.path.basename(x))[0]),
    )
    if max_images:
        paths = paths[:max_images]

    preds = []
    for i, p in enumerate(tqdm(paths)):
        img_id = int(os.path.splitext(os.path.basename(p))[0])
        pil = preprocess_image(Image.open(p).convert("RGB"))
        
        # run predictions for CLIP model and detector
        clip_prob = clip_image_book_prob(pil, clip_processor, clip_model)
        boxes, scores, labels = wrapper.predict_raw(pil, min_threshold)
        
        # add prediction values
        preds.append(
            {
                "image_id": img_id,
                "boxes": boxes,
                "scores": scores,
                "labels": labels,
                "clip_prob": clip_prob,
            }
        )

        # if i < visualize_n:
        #     plt.figure(figsize=(6,6))
        #     plt.imshow(pil)
        #     plt.axis("off")
        #     plt.title(f"id {img_id} | CLIP(book)={clip_prob:.2f}", fontsize=14)
        #     plt.show()

    return preds

# main function for filtering bboxes and returning book amount
def count_from_raw(
    boxes,
    scores,
    labels,
    wrapper_book_id,
    high_th=0.6,
    low_th=0.4,
    min_high=3,
    apply_nms=True,
    nms_iou=0.5,
    clip_prob=None,
    clip_presence_th=CLIP_PRESENCE_TH,
    fallback_low_th=0.15,
    clip_absent_th=0.05,  
):
    # mask out all bboxes with the label "book"
    if wrapper_book_id is not None:
        mask = labels == wrapper_book_id
    else:
        mask = np.ones_like(labels, dtype=bool)

    boxes = boxes[mask]
    scores = scores[mask]

    # if CLIP confidence score for an image is too low, then the image is automatically considered without the books
    if clip_prob is not None and clip_prob < clip_absent_th:
        return 0

    if boxes.shape[0] == 0:
        return 0

    # basic score threshold for bboxes
    high_mask = scores > high_th
    
    # if the amount of highly confident predictions is too low, then we lower our threshold
    if high_mask.sum() >= min_high:
        final_mask = high_mask
    else:
        final_mask = scores > low_th

    boxes_f = boxes[final_mask]
    scores_f = scores[final_mask]

    # if based on the detector threshold there are no books, but CLIP says otherwise we try again and lower score threshold 
    if boxes_f.shape[0] == 0 and (clip_prob is not None and clip_prob >= clip_presence_th):
        retry_mask = scores > fallback_low_th
        boxes_f = boxes[retry_mask]
        scores_f = scores[retry_mask]

    if boxes_f.shape[0] == 0:
        return 0

    # perform nms on resulting bboxes to remove clearly overlapping ones
    if apply_nms:
        keep = perform_nms_np(boxes_f, scores_f, nms_iou)
        return int(len(keep))
    else:
        return int(len(boxes_f))


In [10]:
# run model

# init CLIP
clip_processor = CLIPProcessor.from_pretrained(CLIP_MODEL_ID)
clip_model = CLIPModel.from_pretrained(CLIP_MODEL_ID).to(device)
clip_model.eval()

#init DFine detector
wrapper = DFineWrapper(detector_id)

# create bbox predictions on all images
preds = build_predictions_dataset(wrapper, clip_processor, clip_model, dataset_folder, min_threshold=0.0, max_images=None)
image_ids = [p["image_id"] for p in preds]

y_pred = []
for pred in preds:
    # for each image filter out unneeded bboxes based on their scores and count amount of bboxes with label "book"
    c = count_from_raw(
        pred["boxes"],
        pred["scores"],
        pred["labels"],
        wrapper.book_label_id,
        clip_prob=pred.get("clip_prob", 0.0),
        high_th=DETECTOR_THRESH,
        low_th=DETECTOR_THRESH_LOW
    )
    y_pred.append(c)

# save results as csv
out_df = pd.DataFrame({"image_id": image_ids, "number_of_books": y_pred})
out_df.to_csv("solution.csv", index=False)
print("Saved solution.csv")

  0%|          | 0/1148 [00:00<?, ?it/s]

Saved solution.csv
